In [ ]:
%pip install ../../../

In [ ]:
import torch 

device = torch.device("cuda:7")

In [ ]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [ ]:
from spuco.models import model_factory 
from torch.optim import SGD
from spuco.invariant_train import ERM

model = model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)
erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
erm.train()

In [ ]:
from spuco.group_inference import CorrectNContrastInference

model = model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)
cnc_inference = CorrectNContrastInference(
    trainset=trainset, 
    model=model, 
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    num_epochs=1,
    device=device,
    verbose=True
)

In [ ]:
group_partition = cnc_inference.infer_groups()
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

In [ ]:
from spuco.evaluate import Evaluator 

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
from torch.optim import SGD
from spuco.invariant_train import CorrectNContrastTrain
from spuco.models import model_factory 
from spuco.utils import GroupLabeledDataset

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
cnc_train = CorrectNContrastTrain(
    trainset=GroupLabeledDataset(trainset, group_partition),
    model=model,
    batch_size=512,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    num_pos=10,
    num_neg=10,
    num_epochs=5,
    device=device,
    verbose=True
)
cnc_train.train()

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy

In [ ]:
evaluator.evaluate_spurious_task()